In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Synopsis

In this unit we will learn:

* The basic concept of a linear regression
* How to run a linear regression with statsmodels
* The different capabilities of statsmodels and scikit-learn to run regressions

# Regression Analysis

So far we've covered how you can perform basic data analysis on structured data using Pandas. However, Pandas does not have one of the most common forms of analysis for this type of data built into it --- regression analysis.

Many of you have created a linear regression before, although not all of you may know that you did it. The basic idea is pretty simple, we want to determine the linear relationship between some variable $x$ and another variable $y$. 

Sounds familiar right? All I'm describing is the equation $y = mx + b$ that we learned back in high school. This linear model describes the relationship between some independent $x$ variable and its dependent $y$.

Fitting a regression model is common in many disciplines, so I'm going to show you how to do that in Python now. I want to explain enough of the mathematical background such that everyone can follow along, but this isn't meant as a proper explanation of the underlying statistics or when you should use a regression model. If you're not familiar with this model or the underlying statistics and want to learn more I recommend this [statistics textbook](http://www.amazon.com/Statistics-4th-Edition-David-Freedman/dp/0393929728).

# But...math!

I know, so let's start with a simple, concrete example. Let's say that someone's High-School Senior GPA is almost exactly the same as their first year College GPA. This means that if we wanted to predict a student's first year gpa ($y$) we would guess based on their High school GPA ($x$). More simply, $y = x$.

I'll create that data now with Numpy where I apply some small amount of noise.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#We turn off the latex usage in matplotlib because LaTeX doesn't know
#how to handle a '_' character without it being escaped with a backslash
#Since we use '_' in column names typically this can be a bit of a problem
#If we don't turn this off
mpl.rc('text', usetex=False)

In [ ]:
#I will create 100 gpas, with a random probability
high_school_gpas = np.random.uniform(0, 4, 100)
#The college gpas are created by adding some noise
college_gpas = high_school_gpas + np.random.normal(0, 0.2, 100)

Now let's plot the relationship just so it's clear.

In [ ]:
plt.scatter(high_school_gpas, college_gpas, color='steelblue')
plt.ylabel('College GPA')
plt.xlabel('High School GPA')

Now let's say that we didn't create this data and we wanted to identify the relationship between High School and College GPA. We can do that by fitting a linear model to the data.

With Python we can fit a Linear Model using the `statsmodels` library. Fitting a linear model involves using [Ordinary Least Squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) to identify the parameters. In this lecture I won't go over how this works, but you should look into it later if you plan on fitting linear models.

In [ ]:
import statsmodels.api as sm

First what I do is add a constant to the independent dataset. Without this we are just modelling $y=mx$, instead of $y=mx+b$. Our example doesn't need this constant, but I want to show you how to do it.

In [ ]:
X = sm.add_constant(high_school_gpas)

Now we create the ordinary least squares regression, this is our `model`

In [ ]:
simdata_model = sm.OLS(college_gpas, X)

Once we fit the model, this becomes our result

In [ ]:
simdata_result = simdata_model.fit()

The fitted result has a number of features, but the one that we care about most is the `summary()`. This tells us about our fitted model.

In [ ]:
simdata_result.summary()

Now we can plot the best-fit line along with the data.

In [ ]:
#Here is the data again
plt.scatter(high_school_gpas, college_gpas, color='steelblue')
#Now for our equation
sim_x = np.linspace(0, 4, 100)
sim_y = 1.0262 * sim_x - 0.0608
plt.plot(sim_x, sim_y, color='black', linewidth=2)
#Our labelled axes
plt.ylabel('College GPA')
plt.xlabel('High School GPA')

Pretty darn good!

But this isn't actually real data. I've downloaded some real data that was a part of an introductory statistics textbook. Let's load that csv with Pandas right now.

In [ ]:
import pandas as pd

df = pd.read_csv('../../data/student_gpa_data.csv')

df.head()

In this dataframe we have 10 columns:

    GPA - 1st Year College GPA
    HSGPA - High School GPA
    SATV - SAT Verbal Score
    SATM - SAT Math Score
    Male - Male (1) or Female (0)
    HU - Number of credit hours earned in humanities courses in high school.
    SS - Number of credit hours earned in social science courses in high school.
    FirstGen - First Generation College Student
    White - White (1) or Other Ethnicity (0) 
    CollegeBound - 1 = attended a high school where >=50% students intended to go on to college, 0 = otherwise
    
But right now, let's just look at the relationship between High School GPA and College GPA like we did previously.

Like always we should take a look at the raw data before we try to model it.

In [ ]:
df.plot(kind='scatter', x = 'HSGPA', y = 'GPA', color='steelblue')

Yikes! Looks like my initially generated data was a bit optimistic....

Let's try fitting a linear model to this dataset and see what it looks like.

Now here's a cool thing, there are actually two ways to run regressions with statsmodels. The first one is by importing the vanilla `statsmodels` from `statsmodels.api`. The other way is by calling `statsmodels` formulas, which are from `statmodels.formula.api`. Why is there this crazy naming scheme? ...I can't really tell you actually.

In [ ]:
import statsmodels.formula.api as smf

The great thing about the formula is that it works directly with a Dataframe! We just have to learn a little bit of syntax. I'll show you how it works first.

In [ ]:
realdata_model = smf.ols(formula = 'GPA ~ HSGPA', data = df)

We can type out a linear formula and it will automatically add the constant that we previously had to add manually. When we write a formula it is set up as:

    dependent variable = independent variable
    
but the equals sign is replaced by a `~`. All we need to do is tell it the name of the columns in the dataframe that we want it to use in the model.

We also tell the model that the data is coming from the dataframe.

Now we can fit the model just like before.

In [ ]:
realdata_result = realdata_model.fit()
realdata_result.summary()

### So what does this all mean?

First, we might notice the difference in the slope between the simulated data model line and the real data model line. We can show that here

Remember that the slope of a line is defined as the rise over the run (or $\frac{\Delta y}{\Delta x}$)?

This plot shows that in the fake data for every point increase in High School GPA there was an **equal** point increase in College GPA.

In the real data, the slope of the line is not as steep - meaning that the College GPA *rises* less than the High School GPA *runs*. So if a student had a High School GPA of 4.0, we would predict a College GPA of 3.4. 

In [ ]:
realdata_result.summary()

Next we should notice that this model doesn't explain nearly as much of the real data as the synthetic data. We can tell this by the R-squared value. 

With real data, high school GPA only predicts 20% of the variance in college GPA.

This becomes a bit more intuitive when we plot the regression line against the data.

In [ ]:
#We need to generate our predicted line

real_x = np.arange(2.20, 4.00, 0.1)
real_y = []
for x in real_x:
    real_y.append( realdata_result.params['HSGPA'] * x + realdata_result.params['Intercept'] )

In [ ]:
#Here is the data again
ax = df.plot(kind='scatter', x = 'HSGPA', y = 'GPA', color='steelblue')
#Now for our equation
plt.plot(real_x, real_y, color='black', linewidth=2)
#Our labelled axes
plt.ylabel('College GPA')
plt.xlabel('High School GPA')

We can tell by looking at the data, that there is much more 'spread' or *variability* in the vertical axis than the line accounts for. This makes sense given the definition of the [$R^2$ coefficient](https://en.wikipedia.org/wiki/Coefficient_of_determination). This measure compares the distance from each point to the fitted line against the total variability in the dependent data. We would have $R^2=1.0$ if all of the datapoints sat directly on the predicted line.

This means that in the real world, High School GPA is not solely predictive of a student's College GPA.

I wonder why?

Maybe it's because some students may be bored with regular instruction in High School and don't pay attention or turn in assignments. Maybe if we used both the High School GPA and the SAT scores we would be able to predict the College GPA better.

In [ ]:
test_scores_model = smf.ols(formula='GPA ~ HSGPA + SATV + SATM', data=df)
test_scores_result = test_scores_model.fit()
test_scores_result.summary()

Interesting! Judging by the $R^2$, there was a minor increase in how well this new model fits the data. However, the results weren't quite what we expected!

There is a *small* but *significant* positive relationship between the SAT Verbal score and a student's First Year College GPA. However, there is *no significant* relationship with the SAT Math score.

Guess being good at math only goes so far, huh?

Well, if our SAT test scores only have a very minor effect on our College GPA I wonder what being a first generation college student will do. It's possible that people who are the first in their family to go to college work harder than those of us who had parents go.

In [ ]:
firstgen_model = smf.ols(formula='GPA ~ HSGPA + SATV + FirstGen', data=df)
firstgen_result = firstgen_model.fit()
firstgen_result.summary()

Of course, we could also use the built-in plotting from `statsmodels` too. It will only show a single independent variable at a time, but it's a really quick way to plot!

The first plot we can use is with `plot_fit()`. The `plot_fit()` funciton shows the real data points and the predicted data point (with confidence intervals).

In [ ]:
sm.graphics.plot_fit(firstgen_result, 'HSGPA');

The other plotting function is more of a diagnostic one for users to evaluate how good their model is. It's the `plot_regress_exog` function.

In [ ]:
fig = plt.figure(figsize = (8,8))
sm.graphics.plot_regress_exog(firstgen_result, 'HSGPA', fig=fig);

# Regression methods in Python

Statsmodels has more built into than basic linear regression though. It also has:

    * Generalized Linear Models (Regression with a defined distribution link - logit and probit are in this category)
    * Generalized Estimating Equations (GLM with correlation)
    * Linear Mixed Effects models)
    * Hazard Regression
    * Kernel Regression
    * Generalized Method of Moments
    * Time Series analysis (ARMA, ARIMA, VARMA, VAR)
    
However, if you're looking for some of the newer machine-learning based regression techniques such as:

    * LASSO (Least Absolute Shrinkage and Selection Operator)
    * Ridge Regression
    * Random Forest Regression
    * Support Vector Regression
    
Then you'll need to use `scikit-learn` instead. Scikit Learn also duplicates some of the functions that are contained within `statsmodels` (linear regression, logistic regression, ...., [full list here](http://scikit-learn.org/stable/modules/linear_model.html). One downside of using scikit-learn is that it isn't integrated with Pandas as well as statsmodels (for now, that will likely change in the future).

As an example, we'll show a LASSO regression with scikit-learn.

In [ ]:
df

In [ ]:
from sklearn import linear_model

#First we instantiate the mdoel
clf = linear_model.Lasso(alpha = 0.1)

#Create the X and Y variable sets - must be numpy arrays!
y = df.GPA.values
x = df.loc[:, 'HSGPA':'CollegeBound'].values
#Fit the model
clf.fit(x, y)
#Get the parameters
for param_name, param_value in zip(df.columns[1:-1], clf.coef_):
    print(param_name, param_value)